# **ETAPA 3 - PROCESSAMENTO PARA CAMADA BRONZE**

<br>

---

<br>

Essa etapa será responsável por mover os dados para a camada bronze, colocando em um formato padrão mas sem mais nenhum refinamento, ainda mantendo os dados brutos.

*`Esse é um modelo modular, complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*


<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import gc

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import (
    StructType,
    StructField,
    LongType,
    IntegerType,
    StringType,
    DoubleType,
    DateType,
    BooleanType
)

<br>

---

<br>

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

In [0]:
spark = (
    SparkSession.builder
        .appName("Load Data Bronze")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "128MB")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .getOrCreate()
)

<br>

---

<br>

### Parte 3 - **Definindo Origens, Arquivos e Destinos**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse ` <br>
`--> nome_camada_origem --> nome da camada de origem dos dados` <br>
`--> nome_volume_origem --> nome do volume de origem dos dados dentro da camada` <br>
`--> nome_camada_bronze --> nome da camada de destino dos dados` <br>
`--> nome_volume_bronze --> nome do volume de destino dos dados dentro da camada` <br>

In [0]:
nome_datalakehouse = "dataexperts"
nome_camada_origem = "landing_zone"
nome_volume_origem = "vendas_atual"
nome_camada_bronze = "bronze"
nome_volume_bronze = "vendas"

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_bronze}/{nome_volume_bronze}/"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"""CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_bronze}.{nome_volume_bronze}""")

`Insira no vetor todos os arquivos que deseja fazer esse processamento para camada bronze:` <br>

In [0]:
arquivos = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

<br>

---

<br>

### Parte 4 - **Leitura dos Dados**

Para que seja possível manipular os dados, antes precisamos ler todos eles já com uma estrutura pré-definida.

`Para cada estrutura, é necessário elaborar manualmente como deve ser interpretada`

O código a seguir cria essas estruturas para cada arquivo diferente:

In [0]:
schemas = {
    "categoria_produto.csv": StructType([
    StructField("categoria_id", LongType()),
    StructField("categoria_nome", StringType())
    ]),
    
    "cliente.csv": StructType([
    StructField("cliente_id", LongType()),
    StructField("nome_cliente", StringType()),
    StructField("estado", StringType()),
    StructField("cidade", StringType())
    ]),

    "data.csv": StructType([
    StructField("data_id", LongType()),
    StructField("data", DateType()),
    StructField("ano", IntegerType()),
    StructField("mes", IntegerType()),
    StructField("dia", IntegerType()),
    StructField("dia_semana", StringType()),
    StructField("final_de_semana", BooleanType())
    ]),

    "localidade.csv": StructType([
    StructField("localidade_id", LongType()),
    StructField("estado", StringType()),
    StructField("cidade", StringType())
    ]),

    "produto.csv": StructType([
    StructField("produto_id", LongType()),
    StructField("preco_lista", DoubleType()),
    StructField("categoria_nome", StringType())
    ]),

    "vendas": StructType([
    StructField("venda_id", LongType()),
    StructField("cliente_id", LongType()),
    StructField("produto_id", LongType()),
    StructField("data_id", LongType()),
    StructField("categoria_id", LongType()),
    StructField("localidade_id", LongType()),
    StructField("quantidade", LongType()),
    StructField("preco_lista", DoubleType()),
    StructField("valor_total", DoubleType())
    ])
}

O código a seguir faz a leitura dos dados em cada estrutura, adicionando a coluna nome do arquivo durante a leitura, e armarzenando o resultado desse processo em um vetor de datas frames para que seja possível manipular esses dados.

In [0]:
dataframes = []

for arquivo in arquivos:
    try:
        if arquivo.startswith("vendas"):
            schema = schemas["vendas"]
        else:
            schema = schemas[arquivo]

        dataframe = (
            spark.read
            .option("header", "true")
            .schema(schema)
            .csv(f"{origem_dados}/{arquivo}")
        )

        dataframes.append(dataframe)
        print(f"Leitura do arquivo {arquivo} realizada com sucesso")

    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")

<br>

---

<br>

### Parte 5 - **Salvando os Dados no Formato e Local Corretos**

Por fim, precisamos criar o parquet com os dados que importamos e com a estrutura que construimos anteriormente e mover para camada bronze.

O código a seguir faz essa compressão e envio para o caminho certo (já particionando em ano e mês para ser mais otimizado quando precisarmos utilizar esses dados granularmente):

In [0]:
dimensoes = [
    "bronze_dim_categoria_produto",
    "bronze_dim_cliente",
    "bronze_dim_data",
    "bronze_dim_localidade",
    "bronze_dim_produto"
]

fato = "bronze_fato_vendas"

for dataframe, nome in zip(dataframes[:5], dimensoes):
    dataframe.write.format("delta").mode("overwrite").save(f"{destino_dados}/{nome}")
    print(f"Dimensão {nome} salva com sucesso")

for dataframe in dataframes[5:]:
    dataframe.write.format("delta").mode("append").save(f"{destino_dados}/{fato}")
    print("Fato vendas salvo com sucesso")

> O modo de escrita define como os dados serão gravados no destino:
- ***overwrite***: remove os dados existentes no caminho/partições e grava tudo novamente
- ***append***: adiciona novos dados às partições existentes, sem apagar o que já existe 

> Em pipelines produtivos, 'append' é o mais comum para cargas incrementais. 'overwrite' costuma ser usado apenas em reprocessamentos completos ou ambientes de teste

<br> <br>

---

<br>

### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir exclui todos os Data Frames que podem estar em cache no algoritmo:

In [0]:
"""
No Databricks Free Edition (Serverless), o gerenciamento de cache e memória é feito automaticamente pela plataforma. Por esse motivo, alguns comandos tradicionais do Spark não são suportados, como:

- dataframe.unpersist()
- spark.catalog.clearCache()

A alternativa é remover referências aos DataFrames manualmente e forçar a coleta de lixo do Python quando necessário.
"""

for dataframe in dataframes:
    del dataframe
    
gc.collect()

<br>

---

<br>

### **Resultado Final**

O código a seguir monta um novo Data Frame com os dados diretamente da camada bronze 

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

dataframe_bronze = spark.read.format("delta").load(destino_dados + "/bronze_fato_vendas")

display(dataframe_bronze)

E o código a seguir faz a mesma coisa que o anterior, mas filtrando para mostrar somente mês de janeiro. Mostrando assim a performance mais rápida por ter dividido anteriormente:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

"""
dataframe_bronze_janeiro = (
    spark.read
        .parquet(bronze_destino_dados)
        .filter(
            (col("Ano") == 2012) &
            (col("Mes") == 1)
        )
)

display(dataframe_bronze_janeiro)
"""